In [24]:
import pandas as pd
import numpy as np
import folium
import requests
import time
import json

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
from tqdm.notebook import tqdm 
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [25]:
driver = webdriver.Chrome()
wait_time = 10 # 최대 대기 시간
driver.implicitly_wait(wait_time)
driver.maximize_window() # 윈도우 창 최대화

In [26]:
Urls = {
    "정치": "100",
    "경제": "101",
    "사회": "102",
    "생활/문화": "103",
    "IT/과학": "105",
    "세계": "104"
}
# 모든 카테고리의 기사를 저장할 리스트
all_articles = []

for Urls_name, Url_id in Urls.items():
    # driver = webdriver.Chrome()
    driver.get(f'https://news.naver.com/section/{Url_id}') # 정치 --> 경제 -- >사회 --> 세계
    time.sleep(2)

    html = driver.page_source  # 해당 페이지의 소스 코드 반환
    filename = 'NaverNews.html'
    htmlfile = open(filename, 'w', encoding='UTF-8')
    print(html, file=htmlfile)
    htmlfile.close()
    print(filename + '파일 생성됨')
    soup = BeautifulSoup(html, 'html.parser')

    articles = soup.select('a.sa_text_title')
    
    for article in articles:
        article_link = article['href']  # 기사 링크 추출
        article_title = article.find('strong').get_text()  # 기사 제목 추출

        # 기사 링크에 접근하여 본문과 이미지 가져오기
        driver.get(article_link)
        # time.sleep(2)  # 페이지 로딩 대기

        # 본문과 이미지 가져오기
        article_html = driver.page_source
        article_soup = BeautifulSoup(article_html, 'html.parser')

        # 언론사 추출
        press = article_soup.select_one('meta[property="og:article:author"]')
        press_name = press['content'] if press else '언론사 없음'

        # 본문 추출
        article_body = article_soup.select_one('#dic_area')  # 본문 영역 선택
        article_body_text = article_body.get_text(strip=True) if article_body else '본문 없음'

        # 이미지 추출
        article_images = article_soup.select('#img1')  # 모든 이미지 선택
        image_urls = [img['src'] for img in article_images if 'src' in img.attrs]  # 이미지 URL 추출

        all_articles.append({
            '카테고리': Urls_name,
            '언론사': press_name.split("|")[0].strip(),
            '기사 링크': article_link,
            '기사 제목': article_title,
            '기사 본문': article_body_text[1:],  # 본문 첫 500자 저장
            '이미지 URLs': image_urls
        })

        print(f'카테고리:{Urls_name}')
        print(f'언론사: {press_name}')
        print(f"기사 링크: {article_link}")
        print(f"기사 제목: {article_title}")
        print(f"기사 본문: {article_body_text[1:]}")  # 본문 첫 500자 출력
        print(f"이미지 URLs: {image_urls}")
        print('-' * 30)    

NaverNews.html파일 생성됨
카테고리:정치
언론사: 대전일보 | 네이버
기사 링크: https://n.news.naver.com/mnews/article/656/0000101059
기사 제목: 與, 포털 불공정 개혁 TF 출범…"편향된 뉴스 대안 마련할 것"
기사 본문: 경호 국민의힘 원내대표와 강민국 의원이 12일 '포털 불공정 개혁TF' 임명장 수여식 및 1차 회의에 참석하고 있다. 연합뉴스국민의힘은 12일 네이버·카카오 등 거대 포털의 독점적 지위 남용을 막기 위한 '포털 불공정 개혁 태스크포스(TF)'를 구성했다. TF는 뉴스 제휴시스템의 불공정 이슈, 개인정보·위치정보 등 수집 등 포털의 독점적 지위 남용과 관련한 문제점을 짚어보고 정책적 대안을 마련할 방침이다.TF 위원장인 강민국 의원은 이날 오전 서울 여의도 국회에서 열린 TF 임명장 수여식 및 1차 회의에서 "그동안 대형 포털은 대한민국 경제·사회 분야에서 영향력을 키워왔지만, 이에 부합하는 책임감이 부족하다는 게 국민의 지적"이라며 "사회 각 분야에서 벌어지는 거대 포털의 불공정 행위에 초당적으로 대처하겠다"고 했다.이어 "네이버와 다음의 뉴스 제휴시스템 편향성 문제와 개인·위치 정보의 무분별한 수집을 자세히 살피고 정책 대안을 마련하겠다"고 덧붙였다.추경호 원내대표는 TF 1차 회의에서 "TV나 지면 기사보다 인터넷 포털을 통해 언론 기사를 접속하는 비율이 높아지며 네이버·다음 같은 인터넷 뉴스 서비스와 포털의 사회적 영향력이 점점 커지고 있다"며 "뉴스 포털이 기사 배열에 공정하고 뉴스 공급사와 상생을 도모하며, 이용자의 권익 보호를 우선해 사회적 책임성 제고에 나서야 한다"고 강조했다.한편 TF는 오는 14일 '독과점적 온라인 플랫폼의 책임성 강화 방안'을 주제로 세미나를 열고, 19일에는 네이버 본사 현장 방문을 할 예정이다.
이미지 URLs: ['https://imgnews.pstatic.net/image/656/2024/08/12/0000101059_001_202408121

In [35]:
with open('naver_news_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)

print(f'데이터를 naver_news_data.json 파일로 저장했습니다.')
driver.quit()

데이터를 naver_news_data.json 파일로 저장했습니다.


In [36]:
sbDataFrame = pd.DataFrame(all_articles)
sbDataFrame.columns = ['카테고리','언론사','기사 링크','기사 제목','기사 본문','이미지 URLs']
sbDataFrame.head(n=7)

,카테고리,언론사,기사 링크,기사 제목,기사 본문,이미지 URLs
0,정치,대전일보,https://n.news.naver.com/mnews/article/656/000...,"與, 포털 불공정 개혁 TF 출범…""편향된 뉴스 대안 마련할 것""",경호 국민의힘 원내대표와 강민국 의원이 12일 '포털 불공정 개혁TF' 임명장 수여...,[https://imgnews.pstatic.net/image/656/2024/08...
1,정치,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,"與 '김경수 복권' 갈등에 ""확전 자제"" 공감대…봉합기류속 이견 노출","동훈, '직접 거론' 삼갈듯…친윤-친한은 대리전 양상(서울=연합뉴스) 류미나 안채원...",[https://imgnews.pstatic.net/image/001/2024/08...
2,정치,JTBC,https://n.news.naver.com/mnews/article/437/000...,"윤 대통령, 이명박 전 대통령과 관저서 만찬 진행 중","석열 대통령(왼쪽), 이명박 전 대통령. 〈사진=연합뉴스〉윤석열 대통령이 오늘(12...",[https://imgnews.pstatic.net/image/437/2024/08...
3,정치,머니S,https://n.news.naver.com/mnews/article/417/000...,"정봉주 ""모진 비난 있어도 '이재명팔이' 무리 뿌리 뽑을 것""",봉주 더불어민주당 최고위원 후보가 12일 기자회견을 통해 '이재명팔이' 무리를 뽑겠...,[https://imgnews.pstatic.net/image/417/2024/08...
4,정치,데일리안,https://n.news.naver.com/mnews/article/119/000...,"독립운동가단체들 ""광복절 기념식 불참, 별도 행사 열겠다""",범도기념사업회도 정부 기념식 불참 검토김용만 더불어민주당 의원(오른쪽 여섯 번째)을...,[https://imgnews.pstatic.net/image/119/2024/08...
5,정치,매일신문,https://n.news.naver.com/mnews/article/088/000...,"조국혁신당, 尹 향해 ""내려놓고 술이나 실컷 드세요""","외교안보라인 인사에 술 냄새""기사 이해를 돕기 위한 자료사진. 윤석열 대통령이 대선...",[https://imgnews.pstatic.net/image/088/2024/08...
6,정치,KBS,https://n.news.naver.com/mnews/article/056/001...,김영배 더불어민주당 의원 - 일본 사도광산 ‘강제노동’ 표현 거부…입장은?[전격시사],고성국 : 일본 사도광산의 유네스코 세계유산 등재와 관련해서 정치권 공방이 계속되...,[https://imgnews.pstatic.net/image/056/2024/08...


In [40]:
category = list(sbDataFrame['카테고리'].unique())
print('카테고리 목록 : %d' % len(category))
print(category)

카테고리 목록 : 6
['정치', '경제', '사회', '생활/문화', 'IT/과학', '세계']
